In [1]:
import requests
import json
import requests
from newspaper import Article 
from dotenv import load_dotenv 
import os 

load_dotenv('../.env')
XRapidAPIKey = os.getenv("X-RapidAPI-Key")
xrapidapihost = os.getenv("X-RapidAPI-Host")

import pandas as pd
from dotenv import load_dotenv
import os

# import postgrest
# from postgrest.exceptions import APIError
from tqdm import tqdm
# Supabase
from supabase import create_client, Client
from newspaper import news_pool
# Access the variables
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import newspaper
# Define the URL of the news source you want to scrape
base_url = 'https://www.straitstimes.com/'
TSCMP_url = "https://www.scmp.com/"
reuters_url = "https://www.reuters.com/"
TST_paper = newspaper.build(base_url)
TSCMP = newspaper.build(TSCMP_url)
reuters = newspaper.build(reuters_url)

2023-09-11 10:49:07,095:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.straitstimes.com/feed
2023-09-11 10:49:07,105:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.straitstimes.com/rss
2023-09-11 10:49:07,117:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.straitstimes.com/feeds
2023-09-11 10:49:10,449:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.scmp.com/feeds
2023-09-11 10:49:11,626:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://static.reuters.com/
2023-09-11 10:49:15,464:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.reuters.com/feeds/
2023-09-11 10:49:15,740:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.reuters.com/rss/
2023-09-11 10:49:16,099:CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: https://www.reuters.com/feed/


In [3]:
print(f'Brand of the TST_paper,{TST_paper.brand}, Brand of the TSCMP,{TSCMP.brand}, Brand of the reuters,{reuters.brand}')

Brand of the TST_paper,straitstimes, Brand of the TSCMP,scmp, Brand of the reuters,reuters


Multi-threading article downloads

In [4]:
papers = [TST_paper, TSCMP, reuters]
news_pool.set(papers, threads_per_source=2)
news_pool.join()

2023-09-11 11:26:29,363:WARNING - The following article urls failed the download: https://www.straitstimes.com/business/economy/china-s-gen-z-trying-to-leisure-shop-way-out-of-jobless-blues


In [10]:
papers[0].size(), papers[1].size(), papers[2].size()

(219, 114, 183)

In [14]:
all_url_list = []
for source in papers:
    for article in source.articles:
        all_url_list.append(article.url)

print(f'Number of articles scraped: {len(all_url_list)}')
print(f'Number of distinct articles scraped: {len(set(all_url_list))}')

Number of articles scraped: 516
Number of distinct articles scraped: 516


test one supabase insert

In [14]:
data, count = supabase.table('Article').insert({"Title": "Denmark",'Text':"Text","SourceName":"straitstimes","PublishedDate":"2023-09-04 17:05:53+08:00","ImageUrl":"https://static1.straitstimes.com.sg/s3fs-public/styles/large30x20/public/articles/2023/09/04/afchiangmai300823.jpg","Url":"https://www.straitstimes.com/life/travel/on-the-road-mountain-hopping-in-northern-thailand"}).execute()

2023-09-04 18:01:55,137:INFO - HTTP Request: POST https://exeakzvxpuwdfeavnjgn.supabase.co/rest/v1/Article "HTTP/1.1 201 Created"


In [10]:
for article in TST_paper.articles[:3]:
    # parse the article
    article.parse()
    print(f'Article Title: {article.title}')
    print(f'Article Author: {article.authors}')
    print(f'Article Top Image: {article.top_image}')
    print(f'Aricle SourceName: {TST_paper.brand}')
    print(f'Article Publish Date: {article.publish_date},type: {type(article.publish_date)}')
    print(f'Article Url: {article.url}')

Article Title: On the road: Exploring northern Thailand on motorbike
Article Author: []
Article Top Image: https://static1.straitstimes.com.sg/s3fs-public/styles/large30x20/public/articles/2023/09/04/afchiangmai300823.jpg
Aricle SourceName: straitstimes
Article Publish Date: 2023-09-04 17:20:00+08:00,type: <class 'datetime.datetime'>
Article Url: https://www.straitstimes.com/life/travel/on-the-road-mountain-hopping-in-northern-thailand
Article Title: Paris aims to expand urban cooling system that uses Seine River water
Article Author: []
Article Top Image: https://www.straitstimes.com/themes/custom/straitstimes/favicon.ico
Aricle SourceName: straitstimes
Article Publish Date: 2023-09-04 17:05:53+08:00,type: <class 'datetime.datetime'>
Article Url: https://www.straitstimes.com/world/europe/paris-aims-to-expand-urban-cooling-system-that-uses-seine-river-water
Article Title: One death reported at Burning Man, thousands stranded in mud and rain
Article Author: []
Article Top Image: https:/

# Single source scrape

In [7]:
from Scraper import Scraper
article = Scraper.scrape("https://www.cbsnews.com/news/uk-flight-delays-air-traffic-control-system-technical-problem/")

In [9]:
full_text = article.title + article.text
full_text

'500 flights cancelled as U.K.\'s air traffic control system hit by "nightmare scenario"London — Thousands of travelers faced flight delays and uncertainty Monday after the United Kingdom\'s air traffic control system was hit by technical problems that resulted in the cancellation of at least 500 flights in and out of British airports.  Britain\'s National Air Traffic Service (NATS) said in a statement to CBS News that a technical issue had forced restrictions to the flow of aircraft in and out of the U.K. on Monday, the end of a long weekend and one of the busiest holidays of the year for travel, amid reports of widespread flight delays into London from popular vacation destinations.  Hours later, NATS said it had "identified and remedied" the technical issue and was "now working closely with airlines and airports to manage the flights affected as efficiently as possible." The agency did not say when normal service might be resumed.  BBC News said more than 230 flights departing the U